In [ ]:
!git clone https://github.com/ToelUl/Thermal-Phase-Transition-MC-GPU-Simulation.git

!cp -r Thermal-Phase-Transition-MC-GPU-Simulation/gpu_mc ./

In [ ]:
import time
import matplotlib.pyplot as plt

from gpu_mc import XYModel

import torch

if not torch.cuda.is_available():
    print("⚠️ GPU runtime is not enabled. Please go to 'Runtime' > 'Change runtime type' and set 'Hardware accelerator' to 'GPU'.")
else:
    print("✅ GPU runtime is enabled and ready!")

In [ ]:
L = 32

T = torch.linspace(0.6, 1.5, steps=32)

sampler_xy = XYModel(
    L=L,
    T=T,
    n_chains=30,
    adaptive=False,
    target_acceptance=0.5,
    adapt_rate=0.1,
    device=torch.device("cuda"),
    use_amp=True,
    pt_enabled=True, # Suggestions: Parallel tempering enabled for better sampling.
    )

start = time.time()
samples_xy = sampler_xy(n_sweeps=3000, n_therm=10000, decorrelate=10, pt_interval=10)
end = time.time()
print(f"Elapsed time: {end - start:.2f} s")

In [ ]:
print(f"Samples shape: {samples_xy.shape}")

sampler_xy.spins = samples_xy

energy_xy = sampler_xy.compute_energy().mean(dim=1).cpu().numpy() / L**2
capacity_xy = sampler_xy.compute_heat_capacity().cpu().numpy()
stiffness_xy = sampler_xy.compute_spin_stiffness().cpu().numpy()

plt.figure(figsize=(16, 4))
plt.subplot(131)
plt.plot(T.cpu().numpy(), energy_xy, label='Energy', marker='o')
plt.xlabel('Temperature')
plt.ylabel('Energy')
plt.title('Energy vs. Temperature')
plt.subplot(132)
plt.plot(T.cpu().numpy(), capacity_xy, label='Heat Capacity', marker='o')
plt.xlabel('Temperature')
plt.ylabel('Specific Heat Capacity')
plt.title('Specific Heat Capacity vs. Temperature')
plt.subplot(133)
plt.plot(T.cpu().numpy(), stiffness_xy, label='Spin Stiffness', marker='o')
plt.xlabel('Temperature')
plt.ylabel('Spin Stiffness')
plt.title('Spin Stiffness vs. Temperature')
plt.tight_layout()
plt.show()